# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298362733491                   -0.85    5.2
  2   -8.300189064320       -2.74       -1.25    1.2
  3   -8.300445112917       -3.59       -1.89    2.6
  4   -8.300462144484       -4.77       -2.76    2.4
  5   -8.300464241525       -5.68       -3.06    9.5
  6   -8.300464418259       -6.75       -3.20    2.0
  7   -8.300464528715       -6.96       -3.33    1.4
  8   -8.300464583911       -7.26       -3.45    1.1
  9   -8.300464629148       -7.34       -3.66    1.0
 10   -8.300464636975       -8.11       -3.80    1.0
 11   -8.300464642810       -8.23       -4.05    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67341441061                   -0.70    6.4
  2   -16.67784401853       -2.35       -1.14    1.6
  3   -16.67883802062       -3.00       -1.85    3.1
  4   -16.67901723728       -3.75       -2.43    3.5
  5   -16.67923972560       -3.65       -2.87    6.2
  6   -16.67928573433       -4.34       -3.47    3.2
  7   -16.67928616077       -6.37       -3.73    2.8
  8   -16.67928622083       -7.22       -4.38    2.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32661086700                   -0.56    6.8
  2   -33.33427527584       -2.12       -1.00    1.2
  3   -33.33600236786       -2.76       -1.73    7.1
  4   -33.33689849491       -3.05       -2.57    5.0
  5   -33.33694326883       -4.35       -3.17    5.0
  6   -33.33694386859       -6.22       -3.97    7.1
  7   -33.33694392512       -7.25       -4.52    5.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298384419704                   -0.85    5.4
  2   -8.300244742586       -2.73       -1.58    1.0
  3   -8.300384464701       -3.85       -2.27    5.1
  4   -8.300308910164   +   -4.12       -2.16    5.1
  5   -8.300464304748       -3.81       -3.63    1.8
  6   -8.300464598913       -6.53       -3.94    4.8
  7   -8.300464640039       -7.39       -4.46    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32520807268                   -0.56    7.1
  2   -33.31885276053   +   -2.20       -1.27    1.0
  3   -18.24742225616   +    1.18       -0.48    8.1
  4   -33.27191533711        1.18       -1.47    7.4
  5   -33.10938257117   +   -0.79       -1.29    4.0
  6   -27.91589841576   +    0.72       -0.70    5.4
  7   -33.33558167977        0.73       -2.27    4.8
  8   -33.33378733251   +   -2.75       -2.06    4.1
  9   -33.33388609762       -4.01       -2.20    3.5
 10   -33.33583829155       -2.71       -2.48    2.6
 11   -33.33681921965       -3.01       -2.92    2.1
 12   -33.33689850431       -4.10       -3.15    3.2
 13   -33.33693486801       -4.44       -3.39    3.0
 14   -33.33694108851       -5.21       -3.66    3.0
 15   -33.33694379286       -5.57       -4.07    3.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.